In [1]:
#PySteak Homework
#Import Libraries

import csv
from pathlib import Path

In [2]:
#Load the menu and sales info into list variables, split off headers
# Initializing list objects for the CSVs
# Return the menu and sales 

def load_menu_and_sales_csv():
    #Setting file paths for menu_data and sales_Data
    menu_filepath = Path('Resources/menu_data.csv')
    sales_filepath = Path('Resources/sales_data.csv')
    menu_header = []
    menu = []
    sales_header = []
    sales = []
    
    #Reading the menu_csv into list, splitting off header
    with open(menu_filepath, 'r') as file:
        menu_reader = csv.reader(file, delimiter=',')
        menu_header = next(menu_reader)
        for line in menu_reader:
            menu.append(line)
            
    #Reading the sales_csv into list, splitting off header
    with open(sales_filepath, 'r') as file:
        sales_reader = csv.reader(file, delimiter=',')
        sales_header = next(sales_reader)
        for line in sales_reader:
            sales.append(line)
            
    return menu, sales

In [3]:
#Create a dictionary, with key as each menu item, and value as the report_template dictionary
# Return this dict
def create_total_sales_dict(menu):
    sales_report = {}
    #Initialize the report dict template 
    report_template = {
    '01-count': 0, 
    '02-revenue': 0, 
    '03-cogs': 0, 
    '04-profit': 0, 
    }
    for row in menu:
        item = row[0]
        if item not in sales_report.keys():
            sales_report[item] = dict(report_template)
    
    # Return sales_report dict, fully populated
    return sales_report

In [4]:
#Populate & tabulate quantity from sales data into total sales_report dicts (total info) and date_sales_tracker dicts (daily)
# Return data_tracker_month and date_sales_tracker
def create_daily_sales_dict(sales, sales_report):
    date_tracker_month = []
    date_sales_tracker = {}
    report_template = {
    '01-count': 0, 
    '02-revenue': 0, 
    '03-cogs': 0, 
    '04-profit': 0, 
    }
    
    for row in sales:
        
        # Establish and populate date_tracker_month
        date = row[1].replace("-", "")
        date_month = int(date[-4:] + date[:2])
        if date_month not in date_tracker_month:
            date_tracker_month.append(date_month)
        
        # Convert data from sales records into usable local variables
        date_daily = row[1]
        sales_item = row[4]
        quant = int(row[3])
    
        # Populate the date_sales_tracker with unique days, exclude duplicates
        if date_daily not in date_sales_tracker.keys():
            date_sales_tracker[date_daily] = {}
    
        # Populate the particular days in date_sales_tracker with unique menu item sales, exclude duplicates
        if sales_item not in date_sales_tracker[date_daily].keys():
            placeholder_dict = {}
            placeholder_dict[sales_item] = dict(report_template)
            date_sales_tracker[date_daily].update(placeholder_dict)

        # Populate first the daily sales quantity total, and then the overall sales quantity total
        date_sales_tracker[date_daily][sales_item]["01-count"] += quant
        if sales_item in sales_report.keys():
            sales_report[sales_item]["01-count"] += quant

    return date_tracker_month, date_sales_tracker
    



In [5]:
# Fill in revenue, cogs, profit using sales and relevant info from menu
# Dictionaries are not returned back b/c they are mutable objects and can be modified if passed into function

def populate_revenue_cogs_profit(menu, sales_report, date_sales_tracker):
    for row in menu:

        # Convert data from menu into usable variables
        item = row[0]
        price = float(row[3])
        cost = float(row[4])

        # Calculate 02, 03, and 04 for the overall sales tracker
        if item in sales_report.keys():
            quantity = sales_report[item]["01-count"]
            sales_report[item]["02-revenue"] += (quantity * price)
            sales_report[item]["03-cogs"] += (quantity * cost)
            sales_report[item]["04-profit"] += (quantity * (price - cost))

        # Calculate 02, 03, and 04 for the individual daily sales using nested for loop to loop through 
        # nested dict structure   date_sales_tracked = { 
        #                           "each date of sales" : { 
        #                               "each unique item sold on that date" : { 
        #                                    "01": ,"02": ,"03" : ,"04": 
        for dates in date_sales_tracker:
            if item in date_sales_tracker[dates].keys():
                quantity = date_sales_tracker[dates][item]["01-count"]
                date_sales_tracker[dates][item]["02-revenue"] += (quantity * price)
                date_sales_tracker[dates][item]["03-cogs"] += (quantity * cost)
                date_sales_tracker[dates][item]["04-profit"] += (quantity * (price - cost))

In [6]:
# Make a list of all 0 quantity items in the dict sales_report, then remove them from the sales_report
def clean_up_sales_report(sales_report):
    pop_list = []
    for k in sales_report:
        item = k
        if item in sales_report.keys():
            if sales_report[item]["01-count"] == 0:
                pop_list.append(item)
    for k in pop_list:
        sales_report.pop(k)

In [7]:
# Prepare and publish overall sales output CSV
def prepare_and_publish_total_sales_output_csv(sales_report):
    report_header = ["Item", "Quantity", "Revenue", "COGS", "Profit"]
    total_sales_report_output = Path("total_sales_report.csv")
    with open(total_sales_report_output, "w") as total_sales_report_csv_file:
        csvwriter = csv.writer(total_sales_report_csv_file, delimiter=",")
        csvwriter.writerow(report_header)
        for item in sales_report:
            csvwriter.writerow(
                [
                    item,
                    sales_report[item]["01-count"],
                    sales_report[item]["02-revenue"],
                    sales_report[item]["03-cogs"],
                    sales_report[item]["04-profit"]
                ]
            )

In [8]:
#Print the total months
def print_total_months(date_tracker_month):
    print(f"The number of months in the dataset is: {len(date_tracker_month)}")

In [9]:
#Calculate and print total net profit for each item, and most profitable item
def calculate_and_print_profitability_report(sales_report):
    most_profitable_overall_item = ''
    profit_amount = 0
    for k in sales_report.keys():
        item = k
        profit = sales_report[item]["04-profit"]
        if profit > profit_amount:
            most_profitable_overall_item = k
            profit_amount = profit
        print(f"Item: {k} ... Total Net Profit: ${profit}")
    print()
    print(f"The {most_profitable_overall_item} is the most profitable at ${profit_amount}")
    print()

In [10]:
#Calculate  the most profitable item for each date, and amount sold.  Same for most expensive item
#Total_daily_report_list used for printing daily sales report below
# Returns total_daily_report_list

def total_daily_report_preparation(date_sales_tracker):
    #Initialize list, to be used to hold each individual day's highest profit and highest cost items
    total_daily_report_list =[]
    for dates in date_sales_tracker:
        max_profit_item = ''
        max_profit_item_sales = 0
        max_profit_item_profit = 0
        max_cost_item_name = ''
        max_cost_item_cost = 0
        for a in date_sales_tracker[dates].keys():
            if date_sales_tracker[dates][a]["04-profit"] > max_profit_item_profit:
                max_profit_item_profit = date_sales_tracker[dates][a]["04-profit"]
                max_profit_item_sales = date_sales_tracker[dates][a]["01-count"]
                max_profit_item = a
            if date_sales_tracker[dates][a]["03-cogs"] > max_cost_item_cost:
                max_cost_item_cost = date_sales_tracker[dates][a]["03-cogs"]
                max_cost_item_name = a
        final_daily_report_numbers = [dates, max_profit_item, max_profit_item_profit, max_profit_item_sales, max_cost_item_name, max_cost_item_cost] 
        total_daily_report_list.append(final_daily_report_numbers)
    return total_daily_report_list

In [11]:
# Prepare and publish total_daily_report as an output CSV
def write_and_publish_daily_report_csv(total_daily_report_list):
    max_min_daily_report_header = ["Date", "Max Profit Item", "Profit of MPI", "Sales of MPI", "Max Cost Item", "Sales of MCI"]
    max_min_daily_sales_report_output = Path("max_min_daily_sales_report.csv")
    with open(max_min_daily_sales_report_output, "w", newline='') as max_min_daily_sales_report_csv_file:
        csvwriter = csv.writer(max_min_daily_sales_report_csv_file, delimiter=",")
        csvwriter.writerow(max_min_daily_report_header)
        csvwriter.writerows(total_daily_report_list)

In [12]:
# Print out the daily report list
def print_total_daily_report_list(total_daily_report_list):
    row_counter= 0
    for i in total_daily_report_list:
        print(f"{total_daily_report_list[row_counter][0]}'s {total_daily_report_list[row_counter][1]} = most profitable. Sold {total_daily_report_list[row_counter][3]} for ${total_daily_report_list[row_counter][2]}")
        print(f"{total_daily_report_list[row_counter][0]}'s {total_daily_report_list[row_counter][4]} = most expensive. A total cost of ${total_daily_report_list[row_counter][5]}")
        print()
        row_counter += 1

In [13]:
# Call functions to run the program
def main():
    menu, sales = load_menu_and_sales_csv()
    sales_report = create_total_sales_dict(menu)
    date_tracker_month, date_sales_tracker = create_daily_sales_dict(sales, sales_report)
    populate_revenue_cogs_profit(menu, sales_report, date_sales_tracker)
    clean_up_sales_report(sales_report)
    prepare_and_publish_total_sales_output_csv(sales_report)
    print_total_months(date_tracker_month)
    calculate_and_print_profitability_report(sales_report)
    total_daily_report_list = total_daily_report_preparation(date_sales_tracker)
    write_and_publish_daily_report_csv(total_daily_report_list)
    print_total_daily_report_list(total_daily_report_list)
    
main()

The number of months in the dataset is: 24
Item: nagomi shoyu ... Total Net Profit: $54792.0
Item: shio ramen ... Total Net Profit: $55080.0
Item: spicy miso ramen ... Total Net Profit: $64666.0
Item: vegetarian spicy miso ... Total Net Profit: $64512.0
Item: miso crab ramen ... Total Net Profit: $53340.0
Item: soft-shell miso crab ramen ... Total Net Profit: $63910.0
Item: tori paitan ramen ... Total Net Profit: $64092.0
Item: tonkotsu ramen ... Total Net Profit: $65016.0
Item: burnt garlic tonkotsu ramen ... Total Net Profit: $72560.0
Item: vegetarian curry + king trumpet mushroom ramen ... Total Net Profit: $52944.0
Item: truffle butter ramen ... Total Net Profit: $62874.0

The burnt garlic tonkotsu ramen is the most profitable at $72560.0

01-01-2017's vegetarian spicy miso = most profitable. Sold 38 for $266.0
01-01-2017's truffle butter ramen = most expensive. A total cost of $252.0

01-02-2017's soft-shell miso crab ramen = most profitable. Sold 4 for $28.0
01-02-2017's soft-she